In [1]:
import pandas as pd

In [2]:
import numpy as np

## Lectura de Datos

Se leerán las bases básicas, que solo tienen registros a nivel de usuario. Sólo se leerá la información de campaña que dependa del tiempo. Queda para mejorar, la incorporacion de más informaición temporal. 

In [3]:
train = pd.read_csv("data/raw/ib_base_inicial_train/ib_base_inicial_train.csv")
X_test = pd.read_csv("data/raw/ib_base_inicial_test/ib_base_inicial_test.csv")

sunat = pd.read_csv("data/raw/ib_base_sunat/ib_base_sunat.csv")
reniec = pd.read_csv("data/raw/ib_base_reniec/ib_base_reniec.csv")
vehicular = pd.read_csv("data/raw/ib_base_vehicular/ib_base_vehicular.csv")
campanias = pd.read_csv("data/raw/ib_base_campanias/ib_base_campanias.csv")

In [4]:
train.head()

,codmes,id_persona,codtarget,margen,cem,ingreso_neto,linea_ofrecida
0,201901,165747,0,-5.00,1.0,1781.0,1700
1,201903,100117,0,-5.00,478.0,2106.0,2100
2,201901,65751,1,342.99,687.0,3083.0,4600
3,201901,58047,0,-5.00,182.0,1814.0,3600
4,201901,113221,0,-5.00,582.0,2153.0,4300


## Creación del Target de predicción

Se opta por construir un target binario, para establecer quienes son clientes rentables y, por tanto, es conveniente hacerles campaña para atraerlos. 

In [5]:
y_train = train[['codmes', 'id_persona', 'margen','codtarget']].copy()
y_train["prediction_id"] = y_train["id_persona"].astype(str) + "_" + y_train["codmes"].astype(str)
y_train["target"] = (y_train["margen"] > 0).astype(int)
y_train = y_train.set_index("prediction_id")
X_train = train.drop(["codtarget", "margen"], axis=1)
X_train["prediction_id"] = X_train["id_persona"].astype(str) + "_" + X_train["codmes"].astype(str)
del train

In [6]:
X_train.head()

,codmes,id_persona,cem,ingreso_neto,linea_ofrecida,prediction_id
0,201901,165747,1.0,1781.0,1700,165747_201901
1,201903,100117,478.0,2106.0,2100,100117_201903
2,201901,65751,687.0,3083.0,4600,65751_201901
3,201901,58047,182.0,1814.0,3600,58047_201901
4,201901,113221,582.0,2153.0,4300,113221_201901


In [7]:
y_train.head()

,codmes,id_persona,margen,codtarget,target
prediction_id,,,,,
165747_201901,201901,165747,-5.00,0,0
100117_201903,201903,100117,-5.00,0,0
65751_201901,201901,65751,342.99,1,1
58047_201901,201901,58047,-5.00,0,0
113221_201901,201901,113221,-5.00,0,0


In [8]:
y_train['target_check'] = np.where(y_train['target']==y_train['codtarget'],1,0)

## Consolidación de Bases

Se unene todas las bases por id_persona

In [9]:
sunat.head()

,id_persona,activ_econo,meses_alta
0,174454,Grupo_11,70
1,245154,Grupo_15,27
2,1211,Grupo_07,20
3,146154,Grupo_15,135
4,87697,Grupo_07,55


In [10]:
vehicular.head()

,id_persona,marca,veh_var1,veh_var2
0,14908,ACURA,0.942857,3376.0
1,227555,ADVANCE,0.985714,3426.0
2,211938,AKT,1.000000,3426.0
3,20772,ALESIN,0.985714,3424.0
4,75985,ALESIN,0.985714,3424.0


In [11]:
sunat = sunat.groupby(["id_persona", "activ_econo"]).meses_alta.sum().unstack(level=1, fill_value=0).astype("int32")
vehicular1 = vehicular.groupby(["id_persona", "marca"]).veh_var1.sum().unstack(level=1, fill_value=0).astype("float32")
vehicular2 = vehicular.groupby(["id_persona", "marca"]).veh_var2.sum().unstack(level=1, fill_value=0).astype("float32")
reniec = reniec.set_index("id_persona").astype("float32")
del vehicular

In [12]:
vehicular1.columns = [c + "_v1" for c in vehicular1.columns]
vehicular2.columns = [c + "_v2" for c in vehicular2.columns]

In [13]:
X_train = X_train.set_index("prediction_id").astype("int32").reset_index().set_index("id_persona").join(vehicular1).join(vehicular2).join(reniec).join(sunat)
X_test = X_test.set_index("prediction_id").astype("int32").reset_index().set_index("id_persona").join(vehicular1).join(vehicular2).join(reniec).join(sunat)
del vehicular1, vehicular2, reniec, sunat

In [14]:
camp_canal = campanias.groupby(["codmes", "id_persona", "canal_asignado"]).size().unstack(level=2, fill_value=0).reset_index().set_index("codmes").sort_index().astype("int32")
camp_prod = campanias.groupby(["codmes", "id_persona", "producto"]).size().unstack(level=2, fill_value=0).reset_index().set_index("codmes").sort_index().astype("int32")
del campanias

In [15]:
camp_canal.head()

canal_asignado,id_persona,ABP,AC_PROV,ATM/BPI,BANCA TELEFONICA,BOLSA,BOLSA(E),BOLSA(EX),BOLSA(EX-E),BOLSA(EX-P),...,RED_DE_TIENDAS,Red de Tiendas,S1P,SOMOS_UNO_PROVINCIA,TELEVENTAS,TLV,TLV BD,TLV CD,TLV SEGMENTOS,TLV(EX)
codmes,,,,,,,,,,,,,,,,,,,,,
201801,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201801,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201801,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201801,32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
camp_prod.head()

producto,id_persona,Adelanto de Sueldo,Adquisición Convenios,Adquisición TC,Alcancía,CD Préstamos,CD-Ataque,CD-Defensa,CTS,Cartera ABP,...,Seguro Renta Hospitalaria,Seguro Salud,Seguro Vehicular,Seguro Viajes,Seguro Vida Retorno,Seguros,TELEVENTAS,Trading,Upgrade,Upgrade TC
codmes,,,,,,,,,,,,,,,,,,,,,
201801,5,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,20,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,32,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
import gc
gc.collect()

353

In [18]:
meses = {
    201901: slice(201808, 201810),
    201902: slice(201809, 201811),
    201903: slice(201810, 201812),
    201904: slice(201811, 201901),
    201905: slice(201812, 201902),
    201906: slice(201901, 201903),
    201907: slice(201902, 201904)
}

complementos = []
for mes in meses.keys():
    print("*"*10, mes, "*"*10)
    res = pd.concat([
        camp_canal.loc[meses[mes]].groupby("id_persona").sum(),
        camp_prod.loc[meses[mes]].groupby("id_persona").sum()
        
    ], axis=1)
    res["codmes"] = mes
    res = res.reset_index().set_index(["id_persona", "codmes"]).astype("float32")
    complementos.append(res)

gc.collect()
print("contatenando complementos")
complementos = pd.concat(complementos)
gc.collect()
print("X_train join")
X_train = X_train.reset_index().join(complementos, on=["id_persona", "codmes"]).set_index("prediction_id")
gc.collect()
print("X_test join")
X_test = X_test.reset_index().join(complementos, on=["id_persona", "codmes"]).set_index("prediction_id")
gc.collect()

del camp_canal, camp_prod, complementos,res
gc.collect()

********** 201901 **********
********** 201902 **********
********** 201903 **********
********** 201904 **********
********** 201905 **********
********** 201906 **********
********** 201907 **********
contatenando complementos
X_train join
X_test join


14

In [19]:
X_train.head()

,id_persona,codmes,cem,ingreso_neto,linea_ofrecida,A.N.C._v1,ACS_v1,ACURA_v1,ADVANCE_v1,AGRALE MODASA_v1,...,Seguro Renta Hospitalaria,Seguro Salud,Seguro Vehicular,Seguro Viajes,Seguro Vida Retorno,Seguros,TELEVENTAS,Trading,Upgrade,Upgrade TC
prediction_id,,,,,,,,,,,,,,,,,,,,,
1_201902,1,201902,1140,3945,21600,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1_201904,1,201904,372,2662,9300,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2_201901,2,201901,1,1735,1700,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3_201901,3,201901,322,1569,1500,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3_201902,3,201902,310,1569,1500,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Renombrado de Variables con nombre no ascii

El algoritmo que usamos no se lleva bien con cadenas de texto con caracteres especiales, las renombramos.

In [20]:
non_ascii = X_train.columns[[not all(ord(c) < 128 for c in s) for s in X_train.columns]].tolist()
non_ascii

['LIMA MOTORÂ´S_v1',
 'VICTORIA MOTORÂ´S_v1',
 'VILMENÂ¨S_v1',
 'LIMA MOTORÂ´S_v2',
 'VICTORIA MOTORÂ´S_v2',
 'VILMENÂ¨S_v2',
 'Adquisición Convenios',
 'Adquisición TC',
 'Alcancía',
 'CD Préstamos',
 'Crédito Hipotecario',
 'Crédito Vehicular',
 'Depósito a Plazo',
 'Depósito a Plazo Jubilacion',
 'Depósito a Plazo Renovacion',
 'Déposito a Plazo Renovacion',
 'Membresía',
 'Préstamo Express',
 'Préstamos Personales',
 'Préstamos Reenganche',
 'Retención']

In [21]:
for i, c in enumerate(non_ascii):
    X_train["non_ascii_" + str(i)] = X_train[c]
    X_train = X_train.drop(c, axis= 1)
    X_test["non_ascii_" + str(i)] = X_test[c]
    X_test = X_test.drop(c, axis= 1)

In [22]:
X_train.codmes.unique()

array([201902, 201904, 201901, 201903])

In [24]:
Xt = X_train[X_train.codmes != mes]

## Entrenamiento del Modelo

Se entrena un modelo con valores en default, pero optimizando en nro de estimadores inferiores, con validación basada en meses.

In [52]:
from lightgbm import LGBMClassifier
drop_cols = ["codmes"]
fi = []
test_probs = []
train_probs = []
for mes in X_train.codmes.unique():
    print("*"*10, mes, "*"*10)
    Xt = X_train[X_train.codmes != mes]
    yt = y_train.loc[Xt.index, "target"]
    Xt = Xt.drop(drop_cols, axis=1)

    Xv = X_train[X_train.codmes == mes]
    yv = y_train.loc[Xv.index, "target"]
    
    learner = LGBMClassifier(n_estimators=10000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv.drop(drop_cols, axis=1), yv)], verbose=50)
    
    test_probs.append(pd.Series(learner.predict_proba(X_test.drop(drop_cols, axis=1))[:, -1],
                                index=X_test.index, name="fold_" + str(mes)))
    train_probs.append(pd.Series(learner.predict_proba(Xv.drop(drop_cols, axis=1))[:, -1],
                                index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

********** 201902 **********
Training until validation scores don't improve for 10 rounds.
[50]	training's auc: 0.781409	training's binary_logloss: 0.274736	valid_1's auc: 0.717946	valid_1's binary_logloss: 0.291606
Early stopping, best iteration is:
[76]	training's auc: 0.792861	training's binary_logloss: 0.270832	valid_1's auc: 0.719605	valid_1's binary_logloss: 0.291291
********** 201904 **********
Training until validation scores don't improve for 10 rounds.
[50]	training's auc: 0.770439	training's binary_logloss: 0.274381	valid_1's auc: 0.763971	valid_1's binary_logloss: 0.288995
Early stopping, best iteration is:
[55]	training's auc: 0.773112	training's binary_logloss: 0.273532	valid_1's auc: 0.76424	valid_1's binary_logloss: 0.288845
********** 201901 **********
Training until validation scores don't improve for 10 rounds.
[50]	training's auc: 0.776134	training's binary_logloss: 0.288832	valid_1's auc: 0.737187	valid_1's binary_logloss: 0.247373
Early stopping, best iteration is

## Importancia de Variables

Observamos la importancia media que le dieron los modelos a cada variables

## Optimización de punto de corte

Con las probabilidades calculadas en validación, calcularmos el punto de corte optimo para maximizar la ecuación económica de la empresa

In [54]:
from scipy.optimize import differential_evolution

res = y_train.join(train_probs.rename("probs"))
optimization = differential_evolution(lambda c: -((res.probs > c[0]) * res.margen / res.margen.sum()).sum(), [(0, 1)])
optimization

     fun: -1.0747430437210193
 message: 'Optimization terminated successfully.'
    nfev: 107
     nit: 6
 success: True
       x: array([0.04303723])

## Guardado del modelo para hacer la presentación

In [55]:
test_preds = (test_probs > optimization["x"][0]).astype(int)
test_preds.index.name="prediction_id"
test_preds.name="class"
test_preds.to_csv("benchmark1.csv", header=True)